In [6]:
import os
import csv
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

# CONFIG
INPUT_CSV = r"C:\Users\noahs\Área de Trabalho\DATASET NOAH_KATAMAK\ANALISE_ARTIGO 02\EXPOR_MUNC_UF\EXPORTACOES_CLASSIFICADAS_COMPLETO.csv"
OUTPUT_DIR = r"C:\Users\noahs\Área de Trabalho\DATASET NOAH_KATAMAK\ANALISE_ARTIGO 02\OUTPUT_DETALHADO"
EXCEL_OUT = os.path.join(OUTPUT_DIR, "analise_2018_2025_top5_por_uf_e_ufs_urf.xlsx")
UNIFIED_CSV = os.path.join(OUTPUT_DIR, "unified_selected_rows_2018_2025.csv")
PLOTS_DIR = os.path.join(OUTPUT_DIR, "plots")
TOPK = 5
MIN_YEAR = 2018
MAX_YEAR = 2025

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(PLOTS_DIR, exist_ok=True)

def normalize(s):
    return "" if s is None else s.replace('\ufeff','').strip()

def find_index(header, target):
    t = target.strip().lower()
    for i,h in enumerate(header):
        if normalize(h).lower() == t:
            return i
    return None

def find_any(header, candidates):
    for cand in candidates:
        i = find_index(header, cand)
        if i is not None:
            return i, header[i]
    return None, None

def parse_num(s):
    s = (s or "").strip()
    if s == "":
        return 0.0
    s = s.replace('.', '').replace(',', '.')
    try:
        return float(s)
    except:
        return 0.0

# leitura cabeçalho e detecção flexível de colunas
with open(INPUT_CSV, 'r', encoding='utf-8-sig') as f:
    reader = csv.reader(f, delimiter=';')
    raw_header = next(reader)
    header = [h.strip() for h in raw_header]

# mapeamento flexível: listas de variantes aceitáveis
candidates = {
    'CO_ANO':        ['CO_ANO','ANO','AN0','YEAR'],
    'SG_UF_NCM':     ['SG_UF_NCM','SG_UF_MUN','SG_UF','UF','SG_UF_MUN'],
    'CO_SH4':        ['CO_SH4','SH4','CO_NCM_SH4','CO_NCM','NCM','SH'],
    'VL_FOB':        ['VL_FOB','FOB','VALOR_FOB','VALOR'],
    'KG_LIQUIDO':    ['KG_LIQUIDO','KG','PESO_KG','PESO'],
    'CO_URF':        ['CO_URF','URF','CO_URF_NCM','URF_CODE']
}

idx_map = {}
missing = []
for key, cand_list in candidates.items():
    idx, name = find_any(header, cand_list)
    if idx is None:
        # URF pode faltar; outras são críticas
        if key == 'CO_URF':
            idx_map[key] = None
        else:
            missing.append((key, cand_list))
    else:
        idx_map[key] = idx

if missing:
    raise SystemExit(f"Colunas obrigatórias não encontradas: {missing}. Cabeçalho: {header}")

print("Colunas detectadas (mapa):", {k:(v if v is None else header[v]) for k,v in idx_map.items()})

# estruturas de agregação
sh_by_uf = {}
uf_totals = {}
urf_by_uf = {}  # só preencherá se CO_URF existir

with open(INPUT_CSV, 'r', encoding='utf-8-sig') as f:
    reader = csv.reader(f, delimiter=';')
    next(reader)
    for row in reader:
        # proteção contra linhas curtas
        if len(row) <= max([i for i in idx_map.values() if i is not None]):
            continue
        # ano
        try:
            ano = int(row[idx_map['CO_ANO']].strip())
        except:
            continue
        if ano < MIN_YEAR or ano > MAX_YEAR:
            continue
        uf = row[idx_map['SG_UF_NCM']].strip()
        sh = row[idx_map['CO_SH4']].strip()
        fob = parse_num(row[idx_map['VL_FOB']])
        kg = parse_num(row[idx_map['KG_LIQUIDO']])
        if fob == 0 and kg == 0:
            continue

        uf_totals.setdefault(uf, {'VL_FOB':0.0, 'KG':0.0})
        uf_totals[uf]['VL_FOB'] += fob
        uf_totals[uf]['KG'] += kg

        sh_by_uf.setdefault(uf, {})
        sh_by_uf[uf].setdefault(sh, {'VL_FOB':0.0, 'KG':0.0})
        sh_by_uf[uf][sh]['VL_FOB'] += fob
        sh_by_uf[uf][sh]['KG'] += kg

        # só agrega URF se coluna estiver presente
        if idx_map['CO_URF'] is not None:
            urf = row[idx_map['CO_URF']].strip()
            urf_by_uf.setdefault(uf, {})
            urf_by_uf[uf].setdefault(urf, {'VL_FOB':0.0, 'KG':0.0})
            urf_by_uf[uf][urf]['VL_FOB'] += fob
            urf_by_uf[uf][urf]['KG'] += kg

# identificar tops SH4 por UF
top5_sh_per_uf = {}
for uf, d in sh_by_uf.items():
    df = pd.DataFrame([(sh, v['VL_FOB'], v['KG']) for sh,v in d.items()],
                      columns=['CO_SH4','VL_FOB','KG'])
    top_fob = df.nlargest(TOPK, 'VL_FOB')['CO_SH4'].tolist()
    top_kg = df.nlargest(TOPK, 'KG')['CO_SH4'].tolist()
    top_union = list(dict.fromkeys(top_fob + top_kg))
    top5_sh_per_uf[uf] = {'by_fob': top_fob, 'by_kg': top_kg, 'union': top_union}

uf_totals_df = pd.DataFrame([(uf, v['VL_FOB'], v['KG']) for uf,v in uf_totals.items()],
                            columns=['SG_UF_NCM','VL_FOB','KG'])
top5_ufs_by_fob = uf_totals_df.nlargest(TOPK, 'VL_FOB')['SG_UF_NCM'].tolist()
top5_ufs_by_kg = uf_totals_df.nlargest(TOPK, 'KG')['SG_UF_NCM'].tolist()
top_ufs_union = list(dict.fromkeys(top5_ufs_by_fob + top5_ufs_by_kg))
top2_ufs_by_kg = uf_totals_df.nlargest(2, 'KG')['SG_UF_NCM'].tolist()

print("Top UFs FOB:", top5_ufs_by_fob)
print("Top UFs KG:", top5_ufs_by_kg)
print("Top 2 UFs por KG:", top2_ufs_by_kg)

# 2º passe: extrair linhas originais para seleções
sheet1_rows = []
sheet2_rows = []
sheet3_rows = []
unified_rows = []

with open(INPUT_CSV, 'r', encoding='utf-8-sig') as f:
    reader = csv.reader(f, delimiter=';')
    raw_header = next(reader)
    header = [h.strip() for h in raw_header]
    for row in reader:
        if len(row) <= max([i for i in idx_map.values() if i is not None]):
            continue
        try:
            ano = int(row[idx_map['CO_ANO']].strip())
        except:
            continue
        if ano < MIN_YEAR or ano > MAX_YEAR:
            continue
        uf = row[idx_map['SG_UF_NCM']].strip()
        sh = row[idx_map['CO_SH4']].strip()
        fob = parse_num(row[idx_map['VL_FOB']])
        kg = parse_num(row[idx_map['KG_LIQUIDO']])
        if fob == 0 and kg == 0:
            continue
        row_dict = {header[i]: row[i].strip() if i < len(row) else "" for i in range(len(header))}
        row_dict['CO_SH4'] = sh
        row_dict['source_file'] = os.path.basename(INPUT_CSV)

        if uf in top5_sh_per_uf and sh in top5_sh_per_uf[uf]['union']:
            sheet1_rows.append(row_dict)
            unified_rows.append(row_dict)

        if uf in top_ufs_union:
            topn = set(top5_sh_per_uf.get(uf, {}).get('union', []))
            if sh in topn:
                sheet2_rows.append(row_dict)
                unified_rows.append(row_dict)

# sheet3: se não houver URF não gera essa aba
if idx_map['CO_URF'] is not None:
    for uf in top2_ufs_by_kg:
        urf_dict = urf_by_uf.get(uf, {})
        urf_df = pd.DataFrame([(urf, v['VL_FOB'], v['KG']) for urf,v in urf_dict.items()],
                              columns=['CO_URF','VL_FOB','KG'])
        top5_urf_by_kg = urf_df.nlargest(TOPK, 'KG')
        top5_urf_by_fob = urf_df.nlargest(TOPK, 'VL_FOB')
        for _, r in top5_urf_by_kg.iterrows():
            sheet3_rows.append({'SG_UF_NCM': uf, 'CO_URF': r['CO_URF'], 'VL_FOB': r['VL_FOB'], 'KG': r['KG'], 'METRICA':'KG'})
        for _, r in top5_urf_by_fob.iterrows():
            sheet3_rows.append({'SG_UF_NCM': uf, 'CO_URF': r['CO_URF'], 'VL_FOB': r['VL_FOB'], 'KG': r['KG'], 'METRICA':'FOB'})

df_sheet1 = pd.DataFrame(sheet1_rows)
df_sheet2 = pd.DataFrame(sheet2_rows)
df_sheet3 = pd.DataFrame(sheet3_rows)

# salvar unified CSV
if unified_rows:
    all_keys = list(dict.fromkeys([k for d in unified_rows for k in d.keys()]))
    with open(UNIFIED_CSV, 'w', newline='', encoding='utf-8-sig') as fout:
        writer = csv.DictWriter(fout, fieldnames=all_keys, delimiter=';')
        writer.writeheader()
        seen = set()
        for d in unified_rows:
            key = tuple(d.get(k,"") for k in all_keys)
            if key in seen:
                continue
            seen.add(key)
            writer.writerow(d)
    print("CSV unificado salvo em:", UNIFIED_CSV)
else:
    print("Nenhuma linha selecionada para CSV unificado.")

# salvar Excel com 3 abas (se URF ausente, a aba 3 ficará vazia)
with pd.ExcelWriter(EXCEL_OUT, engine='openpyxl') as xw:
    (df_sheet1 if not df_sheet1.empty else pd.DataFrame()).to_excel(xw, sheet_name='top5_SH4_por_UF', index=False)
    (df_sheet2 if not df_sheet2.empty else pd.DataFrame()).to_excel(xw, sheet_name='top5_SH4_top_UFs', index=False)
    (df_sheet3 if not df_sheet3.empty else pd.DataFrame()).to_excel(xw, sheet_name='top5_URF_top2_ufs_kg', index=False)

print("Excel salvo em:", EXCEL_OUT)

# gerar gráficos para top_ufs_union (visual científico, mais 'juvenil')
for uf in top_ufs_union:
    d = sh_by_uf.get(uf, {})
    if not d:
        continue
    df = pd.DataFrame([(sh, v['VL_FOB'], v['KG']) for sh,v in d.items()],
                      columns=['CO_SH4','VL_FOB','KG'])
    top_fob = df.nlargest(TOPK, 'VL_FOB').set_index('CO_SH4')
    top_kg = df.nlargest(TOPK, 'KG').set_index('CO_SH4')

    for metric, data, suffix in [('VL_FOB', top_fob, 'fob'), ('KG', top_kg, 'kg')]:
        if data.empty:
            continue
        plt.figure(figsize=(9,5))
        ax = data[metric].sort_values().plot(kind='barh')
        ax.grid(axis='x', linestyle='--', linewidth=0.6, alpha=0.7)
        ax.set_xlabel(f"{metric} (soma {MIN_YEAR}-{MAX_YEAR})")
        ax.set_ylabel("SH4")
        ax.set_title(f"{uf} - Top{TOPK} SH4 por {metric} ({MIN_YEAR}-{MAX_YEAR})", fontsize=11, weight='bold')
        for i, v in enumerate(data[metric].sort_values()):
            ax.text(v + v*0.005, i, f"{v:,.0f}", va='center', fontsize=9)
        plt.tight_layout()
        fname = os.path.join(PLOTS_DIR, f"{uf}_top{TOPK}_{suffix}_{MIN_YEAR}_{MAX_YEAR}.png")
        plt.savefig(fname, dpi=200)
        plt.close()

print("Plots salvos em:", PLOTS_DIR)
print("Processo concluído:", datetime.now())


Colunas detectadas (mapa): {'CO_ANO': 'CO_ANO', 'SG_UF_NCM': 'SG_UF_MUN', 'CO_SH4': 'SH4', 'VL_FOB': 'VL_FOB', 'KG_LIQUIDO': 'KG_LIQUIDO', 'CO_URF': None}
Top UFs FOB: ['SP', 'RJ', 'MG', 'PR', 'PA']
Top UFs KG: ['PA', 'MG', 'RJ', 'SP', 'MT']
Top 2 UFs por KG: ['PA', 'MG']
CSV unificado salvo em: C:\Users\noahs\Área de Trabalho\DATASET NOAH_KATAMAK\ANALISE_ARTIGO 02\OUTPUT_DETALHADO\unified_selected_rows_2018_2025.csv
Excel salvo em: C:\Users\noahs\Área de Trabalho\DATASET NOAH_KATAMAK\ANALISE_ARTIGO 02\OUTPUT_DETALHADO\analise_2018_2025_top5_por_uf_e_ufs_urf.xlsx
Plots salvos em: C:\Users\noahs\Área de Trabalho\DATASET NOAH_KATAMAK\ANALISE_ARTIGO 02\OUTPUT_DETALHADO\plots
Processo concluído: 2025-10-10 11:32:57.556901
